## Load DataSet

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "manishjoshimj"
os.environ['KAGGLE_KEY'] = "14910efaaa9fb2b9f03bd5f3dc575116"

!kaggle datasets download -d manishjoshimj/organico-and-recyclabler-dataset

In [ ]:
# Unzip the dataset

!unzip -q organico-and-recyclabler-dataset.zip -d waste-dataset

## Necessary Imports

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import random

In [ ]:
# Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
DATASET_DIR = '/content/waste-dataset/O_R_WASTE_CLASSIFICATION_DATASET'


In [ ]:
# Transforming the dataset using randomization

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Loading Dataset

train_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "TRAIN"), transform=train_transform)
test_dataset = datasets.ImageFolder(os.path.join(DATASET_DIR, "TEST"), transform=test_transform)